## Шаг 1-2. Развёртывание Trino и PostgreSQL
**Параметры:**
- **Trino UI:** http://localhost:8081
- **PostgreSQL:** localhost:5432
- **Каталог Trino:** `postgresql`

## Шаг 3. Подключение к Trino из Python

In [1]:
!pip install trino pandas --quiet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from trino.dbapi import connect
import pandas as pd

conn = connect(
    host="localhost",
    port=8081,
    user="admin",
    catalog="postgresql",
    schema="public"
)

print("Connected to trino database")

Connected to trino database


In [3]:
cursor = conn.cursor()
cursor.execute("SHOW CATALOGS")
catalogs = cursor.fetchall()
print("Available Catalogs:")
for c in catalogs:
    print(f"  - {c[0]}")

Available Catalogs:
  - postgresql
  - system


In [4]:
cursor.execute("SHOW TABLES FROM postgresql.public")
tables = cursor.fetchall()
print("Tables in postgresql.public:")
for t in tables:
    print(f"- {t[0]}")

Tables in postgresql.public:
- students


## Шаг 4. SQL-запрос к PostgreSQL через Trino

In [5]:
cursor.execute("SELECT * FROM postgresql.public.students")
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=columns)
print(f"Got {len(df)} rows")
df

Got 5 rows


,id,name,course,score,enrollment_date,is_active
0,1,Иванов Иван,Data Engineering,85.50,2024-09-01,True
1,2,Петрова Мария,Machine Learning,92.00,2024-09-01,True
2,3,Сидоров Алексей,Data Engineering,78.30,2024-09-15,True
3,4,Козлова Анна,DevOps,88.70,2024-10-01,False
4,5,Смирнов Дмитрий,Machine Learning,95.20,2024-09-01,True


In [6]:
cursor.execute("""
    SELECT 
        course,
        COUNT(*) as students_count,
        ROUND(AVG(score), 2) as avg_score
    FROM postgresql.public.students
    WHERE is_active = true
    GROUP BY course
    ORDER BY avg_score DESC
""")

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

df_agg = pd.DataFrame(rows, columns=columns)
print("Mean students marks (active studs):")
df_agg

Mean students marks (active studs):


,course,students_count,avg_score
0,Machine Learning,2,93.60
1,Data Engineering,2,81.90


## Шаг 5. Вывод результатов

In [7]:
print("All studs:")
display(df)

print("\nStats:")
display(df_agg)

All studs:


,id,name,course,score,enrollment_date,is_active
0,1,Иванов Иван,Data Engineering,85.50,2024-09-01,True
1,2,Петрова Мария,Machine Learning,92.00,2024-09-01,True
2,3,Сидоров Алексей,Data Engineering,78.30,2024-09-15,True
3,4,Козлова Анна,DevOps,88.70,2024-10-01,False
4,5,Смирнов Дмитрий,Machine Learning,95.20,2024-09-01,True



Stats:


,course,students_count,avg_score
0,Machine Learning,2,93.60
1,Data Engineering,2,81.90


In [8]:
cursor.close()
conn.close()
print("Conn closed")

Conn closed
